In [1]:
# CALLING LIBRARIES
import pandas as pd
import numpy as np
import os
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder as onehot
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier 
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [2]:
#MOUNTING G DRIVE
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# SETTING UP PATH OF ZIP FILE
path =  "/content/drive/MyDrive/" +"Loan_default_train_v2.csv.zip"



In [4]:
#READING CSV FILE SHOWING INCLUION OF LOSS KEY
train = pd.read_csv(
                     path,
                     header=0,  # First row is header-row
                    )

train.dtypes


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (135,204,274,417) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


id        int64
f1        int64
f2        int64
f3      float64
f4        int64
         ...   
f775    float64
f776      int64
f777      int64
f778      int64
loss      int64
Length: 771, dtype: object

In [5]:
abc=train.copy()
print(train.shape)    #Number of rows and columns respectively
print()
train.tail(10)



    

(105471, 771)



,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
105461,105462,132,8,0.798622,2200,4,76973,9968.0,1736.0,130.93,131.95,7,0.7949,0.7381,13881640,0.7623,0.7154,0.5492,0.7154,0.7881,0.7881,2.986603e+09,93,76,1141668.0,93.0,93.0,114.63,102.92,93.0,0.9588,0,0,0,9,0,0,0.70770,0.70386,0.726767,...,-5.47,10,58.94,-223.79,1231.75,2.3653,-0.9950,2.6952,6.0727,5.0287,4.1865,0.060313,3.3633,2.8733,1.7230,1.0748,0.110862,5.7404,4.6416,3.7842,0.072331,7.9671,7.6709,-0.2119,1,-0.55,-0.475,-0.515,-0.5622,-5.06,9,3.23,-2.27,1.66,0.2069,-0.6387,0,0,93,0
105462,105463,139,8,0.771847,2200,4,8563,8452.0,1695.0,136.11,138.87,7,0.8644,0.8430,3124217,0.8160,0.8160,0.7346,0.8644,0.9107,0.8718,3.149728e+09,102,86,1446156.0,102.0,102.0,134.78,116.76,102.0,0.9714,0,0,0,20,0,0,0.78800,0.81440,0.782310,...,25.91,21,464.96,2807.32,35986.23,4.5408,0.8400,7.6474,14.8798,12.9115,11.2416,0.056517,8.4177,8.0331,5.2032,3.4096,0.080651,14.3499,12.2164,10.4299,0.050652,17.2504,16.5146,-0.0893,1,-0.43,-0.465,-0.457,-0.5650,-11.30,20,7.95,-5.93,4.58,0.2798,0.9106,0,0,93,0
105463,105464,136,8,0.420418,2200,4,3793,7820.0,11541.0,132.08,135.02,7,0.7377,0.7377,2555997,0.7200,0.7200,0.5143,0.7258,0.7759,0.7759,2.227500e+09,90,83,1012500.0,90.0,90.0,101.56,95.43,90.0,0.9783,0,0,0,19,0,0,0.75000,0.72800,0.772353,...,3.43,20,366.66,293.49,32317.67,4.2783,0.4063,6.9778,14.1944,12.3686,10.8258,0.066413,8.0599,7.0225,4.3821,2.7692,0.086597,13.4683,11.4289,9.7408,0.063458,16.9464,16.4027,-0.2241,1,-0.50,-0.460,-0.480,-0.6216,-11.81,19,8.26,-6.08,4.60,0.2199,0.9908,0,0,93,3
105464,105465,127,8,0.839374,2200,4,7458,7548.0,3440.0,131.74,128.19,7,0.7966,0.7966,2682000,0.8034,0.7705,0.6447,0.7899,0.8319,0.8319,2.431844e+09,94,74,1228204.0,94.0,94.0,115.86,104.03,94.0,0.9792,0,0,0,10,0,0,0.78525,0.77345,0.775400,...,41.03,11,414.52,4127.37,46710.40,4.8755,0.7909,3.6517,7.2566,6.1982,5.3035,0.036136,3.8035,3.5885,2.1846,1.3413,0.064853,7.1744,6.0893,5.1751,0.031294,8.6384,8.2572,-0.1681,1,-0.43,-0.355,-0.407,-0.5340,-5.34,10,3.53,-2.54,1.90,0.2605,0.7329,0,0,93,0
105465,105466,140,8,0.888521,2200,4,9836,6377.0,4286.0,135.88,139.08,7,0.8814,0.8455,69314768,0.8525,0.8000,0.6764,0.8000,0.8814,0.8814,3.712051e+09,104,89,1406080.0,104.0,104.0,137.43,119.05,104.0,0.9811,0,0,0,3,0,0,0.75000,0.79000,0.732367,...,8.92,4,68.74,343.84,2037.66,3.4946,-0.5429,0.9080,2.0385,1.6993,1.4258,0.073143,1.0094,1.0138,0.6265,0.3968,0.127793,1.9480,1.5902,1.3081,0.076217,2.5016,2.3851,-0.1186,1,-0.81,-0.655,-0.809,-0.6000,-1.80,3,1.42,-1.17,0.98,0.3367,0.9228,0,0,93,0
105466,105467,126,8,0.006620,2200,4,76831,331.0,2366.0,125.25,125.80,7,0.4878,0.4878,9281480,0.4918,0.4615,0.2147,0.4615,0.4800,0.4800,1.178496e+09,60,0,446400.0,60.0,60.0,73.96,66.40,60.0,0.5607,0,0,0,3,0,0,0.62075,0.49535,0.677967,...,-0.67,4,2.45,-0.30,2.20,0.7645,0.7937,0.8106,1.6156,1.2662,1.0186,0.167967,0.8336,0.8120,0.4839,0.2955,0.190560,1.5766,1.2164,0.9619,0.161290,1.9770,1.8424,-0.5200,1,0.00,-0.350,-0.050,-0.4600,-1.38,3,1.07,-0.92,0.82,0.3809,-0.5489,0,0,93,0
105467,105468,137,9,0.358568,2200,4,9836,3898.0,1106.0,135.88,136.99,10,0.7227,0.7049,901745,0.6880,0.6880,0.5147,0.6935,0.7818,0.7478,2.206863e+09,86,50,1013252.0,86.0,86.0,113.64,98.44,86.0,0.8776,0,0,0,3,0,0,0.64400,0.68920,0.623833,...,9.75,4,34.31,129.52,516.69,1.6236,-0.6855,0.5262,1.5442,1.1484,0.8691,0.117873,0.5842,0.6314,0.3230,0.1701,0.142930,1.4767,1.0764,0.7993,0.118383,1.7019,1.4648,-0.2182,1,-0.14,-0.210,-0.176,-0.2333,-0.70,3,0.33,-0.18,0.10,0.2357,-0.8627,1,0,93,0
105468,105469,129,9,0.241858,2200,4,627,7081.0,6577.0,128.44,128.99,10,0.7727,0.7658,12380709,0.7328,0.7328,0.5270,0.7328,0.7798,0.7522,2.469866e+09,85,52,932025.0,85.0,85.0,112.32,97.30,85.0,0.8500

In [6]:
#CLEANING DATA
train.isnull().sum()

id         0
f1         0
f2         0
f3         0
f4         0
        ... 
f775    1525
f776       0
f777       0
f778       0
loss       0
Length: 771, dtype: int64

In [7]:
train.dropna(inplace =True)
train.shape                   #ALMOST HALF ROWS REMOVED DUE TO PRESENCE OF NULL VALUES


(51940, 771)

In [8]:
print(train.info())   # We can observe "object" datatypes(19) of keys(columns) and try to remove them

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51940 entries, 2 to 105468
Columns: 771 entries, id to loss
dtypes: float64(653), int64(99), object(19)
memory usage: 305.9+ MB
None


In [9]:
Object_removal = train.select_dtypes(include=['object'])
print(Object_removal.head(20))

# FOR loop for checking datatype of object type and removing them

for i in train.select_dtypes(include=['object']).columns:
    train.drop(labels=i, axis=1, inplace=True)

                  f137  ...                   f698
2      186000000000000  ...   19855991371293999104
4          52152926246  ...   94766610066210996224
6      226000000000000  ...  221765441912190009344
7    92100000000000000  ...  131851525291529994240
8       61300000000000  ...   46207102369444003840
9        6280000000000  ...   35263226155725000704
10       6280000000000  ...   25152340930400002048
11     226000000000000  ...  209628510163069992960
13    3860000000000000  ...  154252555488300007424
14      61300000000000  ...  155579283901710008320
15    9550000000000000  ...  133605068451669999616
16    3120000000000000  ...  167388960777530015744
17       2640000000000  ...  138744296147760005120
18  120000000000000000  ...   46443482867865001984
19       2250000000000  ...   42176521116234997760
21       5550000000000  ...   14827763877915000832
25       6280000000000  ...  101009490659550003200
26     142000000000000  ...   93214986789537005568
27       2250000000000  ...  20

In [10]:

print(train.shape)   # DATA ready for splitting into testing and training where further model can be applied
print(train.info())

(51940, 752)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 51940 entries, 2 to 105468
Columns: 752 entries, id to loss
dtypes: float64(653), int64(99)
memory usage: 298.4 MB
None



 MODELING

In [11]:
#DATA SPLITTING 
X = train.iloc[:,1:751]
Y = train.iloc[:,751]                   # Selection of Loss Column as hyperparameter
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)    #20 percent proportion of data in testing data split

In [12]:
# APPLYING RANDOM FOREST MODEL
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=50)
model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
model.score(X_test,Y_test)    #Accuracy of the model determined which comes close to 90%

0.9002695417789758

In [14]:

mse = mean_squared_error(Y_test, model.predict(X_test))   #RMSE which comes near 15% 
print("The mean squared error (MSE) on test set: {}".format(mse))

The mean squared error (MSE) on test set: 15.293415479399307
